In [1]:
import pandas as pd
import numpy as np

In [2]:
# how can I put the names into a list and use a function to generate the dataframes (along with the correct names)?

master_df = pd.read_csv('Master.csv')
pitching_df = pd.read_csv('Pitching.csv')
batting_df = pd.read_csv('Batting.csv')
fielding_df = pd.read_csv('Fielding.csv')
awards_df = pd.read_csv('AwardsPlayers.csv')
allstar_df = pd.read_csv('AllstarFull.csv')
hof_df = pd.read_csv('HallOfFame.csv')
appearances_df = pd.read_csv('Appearances.csv')

In [3]:
pd.set_option('display.max_columns', 85)
pd.set_option('display.max_rows', 85)

In [4]:
#batting_df.head()

In [5]:
#batting_df['stint'].value_counts()

In [6]:
years_played = batting_df.groupby('playerID').nunique()
years_played = years_played[['yearID']].rename(columns = {'yearID': 'years_played'})
years_played

,years_played
playerID,
aardsda01,9
aaronha01,23
aaronto01,7
aasedo01,13
abadan01,3
...,...
zupofr01,3
zuvelpa01,9
zuverge01,8


In [8]:
batting_career = batting_df.drop(['yearID', 'stint', 'teamID', 'lgID'], axis=1).groupby(by = 'playerID').sum()
#batting_career

In [9]:
fielding_career = fielding_df[['playerID', 'G', 'GS', 'PO', 'A', 'E', 'DP']].groupby(by = 'playerID').sum()
#fielding_career

In [10]:
awards_career = pd.pivot_table(awards_df, values = 'yearID', index = 'playerID', columns = 'awardID', aggfunc='count')
#awards_career

In [11]:
years_allstar = allstar_df.groupby('playerID').nunique()
years_allstar = years_allstar[['yearID']].rename(columns = {'yearID': 'years_allstar'})

In [27]:
filt = (hof_df['inducted'] == 'Y') & (hof_df['category'] == 'Player') 
hof_df = hof_df[filt]

In [13]:
career_appearances = appearances_df.drop(['yearID', 'teamID', 'lgID'], axis=1).groupby(by = 'playerID').sum()
career_appearances['prop_games_pitcher'] = career_appearances.G_p/career_appearances.G_all
career_appearances['prop_games_catcher'] = career_appearances.G_c/career_appearances.G_all
#career_appearances

In [35]:
career = pd.merge(years_played, batting_career, how = 'inner', on = 'playerID'
                 ).merge(fielding_career, how = 'inner', on = 'playerID'
                        ).merge(awards_career, how = 'left', on = 'playerID'
                               ).merge(years_allstar, how = 'left', on = 'playerID'
                                      ).merge(hof_df, how = 'left', on = 'playerID'
                                             ).merge(career_appearances, how = 'inner', on = 'playerID'
                                                    ).merge(master_df, how = 'inner', on = 'playerID')
career

,playerID,years_played,G_x,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,G_y,GS_x,PO,A,E,DP,ALCS MVP,All-Star Game MVP,Babe Ruth Award,Baseball Magazine All-Star,Branch Rickey Award,Comeback Player of the Year,Cy Young Award,Gold Glove,Hank Aaron Award,Hutch Award,Lou Gehrig Memorial Award,Most Valuable Player,NLCS MVP,Outstanding DH Award,Pitching Triple Crown,Reliever of the Year Award,Roberto Clemente Award,...,G_all,GS_y,G_batting,G_defense,G_p,G_c,G_1b,G_2b,G_3b,G_ss,G_lf,G_cf,G_rf,G_of,G_dh,G_ph,G_pr,prop_games_pitcher,prop_games_catcher,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,deathCountry,deathState,deathCity,nameFirst,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID
0,aardsda01,9,331,4,0,0,0,0,0,0.0,0.0,0.0,0,2.0,0.0,0.0,1.0,0.0,0.0,331,0.0,11,29.0,3.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,331,0.0,139,331,331,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,1.000000,0.0000,1981.0,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,NaN,NaN,NaN,David,Aardsma,David Allan,215.0,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01
1,aaronha01,23,3298,12364,2174,3771,624,98,755,2297.0,240.0,73.0,1402,1383.0,293.0,32.0,21.0,121.0,328.0,3020,2977.0,7436,429.0,144.0,218.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,...,3298,3173.0,3298,2985,0,0,210,43,7,0,315,308,2174,2760,201.0,122.0,1.0,0.000000,0.0000,1934.0,2.0,5.0,USA,AL,Mobile,NaN,NaN,NaN,NaN,NaN,NaN,Hank,Aaron,Henry Louis,180.0,72.0,R,R,1954-04-13,1976-10-03,aaroh101,aaronha01
2,aaronto01,7,437,944,102,216,42,6,13,94.0,9.0,8.0,86,145.0,3.0,0.0,9.0,6.0,36.0,387,206.0,1317,113.0,22.0,124.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,437,206.0,437,346,0,0,232,7,10,0,135,1,2,137,0.0,102.0,35.0,0.000000,0.0000,1939.0,8.0,5.0,USA,AL,Mobile,1984.0,8.0,16.0,USA,GA,Atlanta,Tommie,Aaron,Tommie Lee,190.0,75.0,R,R,1962-04-10,1971-09-26,aarot101,aaronto01
3,aasedo01,13,448,5,0,0,0,0,0,0.0,0.0,0.0,0,3.0,0.0,0.0,0.0,0.0,0.0,448,91.0,67,135.0,13.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,448,91.0,81,448,448,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,1.000000,0.0000,1954.0,9.0,8.0,USA,CA,Orange,NaN,NaN,NaN,NaN,NaN,NaN,Don,Aase,Donald William,190.0,75.0,R,R,1977-07-26,1990-10-03,aased001,aasedo01
4,abadan01,3,15,21,1,2,0,0,0,0.0,0.0,1.0,4,5.0,0.0,0.0,0.0,0.0,1.0,9,4.0,37,1.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,15,4.0,15,9,0,0,8,0,0,0,0,0,1,1,0.0,7.0,1.0,0.000000,0.0000,1972.0,8.0,25.0,USA,FL,Palm Beach,NaN,NaN,NaN,NaN,NaN,NaN,Andy,Abad,Fausto Andres,184.0,73.0,L,L,2001-09-10,2006-04-13,abada001,abadan01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18706,zupofr01,3,16,18,3,3,1,0,0,0.0,0.0,0.0,2,6.0,0.0,0.0,0.0,0.0,0.0,13,1.0,31,1.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,16,1.0,16,13,0,13,0,0,0,0,0,0,0,0,0.0,5.0,0.0,0.000000,0.8125,1939.0,8.0,29.0,USA,CA,San Francisco,2005.0,3.0,25.0,USA,CA,Burlingame,Frank,Zupo,Frank Joseph,182.0,71.0,L,R,1957-07-01,1961-05-09,zupof101,zupofr01
18707,zuvelpa01,9,209,491,41,109,17,2,2,20.0,2.0,0.0,34,50.0,1.0,2.0,18.0,0.0,8.0,201,136.0,267,415.0,23.0,84.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,209,138.0,209,197,0,0,0,55,14,133,0,0,0,0,3.0,14.0,9.0,0.000000,0.0000,1958.0,10.0,31.0,USA,CA,San Mateo,NaN,NaN,NaN,NaN,NaN,NaN,Paul,Zuvella,Paul,173.0,72.0,R,R,1982-09-04,1991-05-02,zuvep001,zuvelpa01
18708,zuverge01,8,266,142,5,21,2,1,0,7.0,0.0,1.0,9,39.0,0.0,0.0,16.0,0.0,3.0,265,31.0,45,145.0,7.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,266,31.0,266,265,265,0,0

In [36]:
filt = (career.prop_games_pitcher < 0.10) & (career.prop_games_catcher < 0.10)

In [37]:
infoutfielders = career[filt]
infoutfielders

,playerID,years_played,G_x,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,G_y,GS_x,PO,A,E,DP,ALCS MVP,All-Star Game MVP,Babe Ruth Award,Baseball Magazine All-Star,Branch Rickey Award,Comeback Player of the Year,Cy Young Award,Gold Glove,Hank Aaron Award,Hutch Award,Lou Gehrig Memorial Award,Most Valuable Player,NLCS MVP,Outstanding DH Award,Pitching Triple Crown,Reliever of the Year Award,Roberto Clemente Award,...,G_all,GS_y,G_batting,G_defense,G_p,G_c,G_1b,G_2b,G_3b,G_ss,G_lf,G_cf,G_rf,G_of,G_dh,G_ph,G_pr,prop_games_pitcher,prop_games_catcher,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,deathCountry,deathState,deathCity,nameFirst,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID
1,aaronha01,23,3298,12364,2174,3771,624,98,755,2297.0,240.0,73.0,1402,1383.0,293.0,32.0,21.0,121.0,328.0,3020,2977.0,7436,429.0,144.0,218.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,...,3298,3173.0,3298,2985,0,0,210,43,7,0,315,308,2174,2760,201.0,122.0,1.0,0.0,0.0,1934.0,2.0,5.0,USA,AL,Mobile,NaN,NaN,NaN,NaN,NaN,NaN,Hank,Aaron,Henry Louis,180.0,72.0,R,R,1954-04-13,1976-10-03,aaroh101,aaronha01
2,aaronto01,7,437,944,102,216,42,6,13,94.0,9.0,8.0,86,145.0,3.0,0.0,9.0,6.0,36.0,387,206.0,1317,113.0,22.0,124.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,437,206.0,437,346,0,0,232,7,10,0,135,1,2,137,0.0,102.0,35.0,0.0,0.0,1939.0,8.0,5.0,USA,AL,Mobile,1984.0,8.0,16.0,USA,GA,Atlanta,Tommie,Aaron,Tommie Lee,190.0,75.0,R,R,1962-04-10,1971-09-26,aarot101,aaronto01
4,abadan01,3,15,21,1,2,0,0,0,0.0,0.0,1.0,4,5.0,0.0,0.0,0.0,0.0,1.0,9,4.0,37,1.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,15,4.0,15,9,0,0,8,0,0,0,0,0,1,1,0.0,7.0,1.0,0.0,0.0,1972.0,8.0,25.0,USA,FL,Palm Beach,NaN,NaN,NaN,NaN,NaN,NaN,Andy,Abad,Fausto Andres,184.0,73.0,L,L,2001-09-10,2006-04-13,abada001,abadan01
6,abadijo01,1,12,49,4,11,0,0,0,5.0,1.0,0.0,0,3.0,0.0,0.0,0.0,0.0,0.0,12,0.0,129,3.0,13.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12,0.0,12,12,0,0,12,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1850.0,11.0,4.0,USA,PA,Philadelphia,1905.0,5.0,17.0,USA,NJ,Pemberton,John,Abadie,John W.,192.0,72.0,R,R,1875-04-26,1875-06-10,abadj101,abadijo01
7,abbated01,9,855,3044,355,772,99,43,11,324.0,142.0,0.0,289,16.0,0.0,33.0,93.0,0.0,0.0,830,0.0,1872,2368.0,315.0,247.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,855,0.0,855,855,0,0,0,419,20,388,0,2,1,3,0.0,0.0,0.0,0.0,0.0,1877.0,4.0,15.0,USA,PA,Latrobe,1957.0,1.0,6.0,USA,FL,Fort Lauderdale,Ed,Abbaticchio,Edward James,170.0,71.0,R,R,1897-09-04,1910-09-15,abbae101,abbated01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18701,zuberjo01,2,68,136,13,34,7,1,3,16.0,1.0,0.0,12,20.0,1.0,1.0,1.0,1.0,4.0,31,26.0,167,12.0,2.0,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,68,26.0,68,31,0,0,26,0,0,0,5,0,0,5,0.0,39.0,0.0,0.0,0.0,1969.0,12.0,10.0,USA,CA,Encino,NaN,NaN,NaN,NaN,NaN,NaN,Jon,Zuber,Jon Edward,190.0,73.0,L,L,1996-04-19,1998-09-27,zubej001,zuberjo01
18702,zuletju01,2,79,174,24,43,11,0,9,36.0,0.0,2.0,10,51.0,1.0,6.0,0.0,1.0,5.0,55,36.0,296,15.0,5.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,79,36.0,79,55,0,0,49,0,0,0,6,0,0,6,0.0,37.0,1.0,0.0,0.0,1975.0,3.0,28.0,Panama,Panama,Panama,NaN,NaN,NaN,NaN,NaN,NaN,Julio,Zuleta,Julio Ernesto,230.0,78.0,R,R,2000-04-06,2001-06-25,zulej001,zuletju01
18705,zupcibo01,4,319,795,99,199,47,4,7,80.0,7.0,5.0,57,137.0,3.0,6.0,20.0,8.0,15.0,285,198.0,483,22.0,12.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,319,198.0,319,285,0,0,1,0,2

In [ ]:
career = pd.merge(batting_career, awards_career, how = 'left', on = 'playerID')#.fillna(0)

In [ ]:
career

In [ ]:
career = pd.merge(batting_career, years_played,
                  how = 'inner', on = 'playerID')

In [ ]:
career

In [ ]:
master_df